In [12]:
from classes import *
import numpy as np
import networkx as nx
import pylab as plt
from scipy.stats import norm,uniform,binom,poisson
import random
from functools import reduce
from itertools import product
from collections import deque
import re
from time import sleep
import json

In [2]:
import requests as req
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from webdriver_manager.firefox import GeckoDriverManager

PATH=FirefoxService(GeckoDriverManager().install())
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\Firefox.exe')

opciones=Options()

[WDM] - Downloading: 16.9kB [00:00, 8.64MB/s]                   
[WDM] - Downloading: 16.9kB [00:00, 8.64MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 32.6MB/s]


In [3]:
line_dict = {}

def table_extractor(k):
    driver = webdriver.Firefox(firefox_binary=binary,options = opciones)
    url = 'https://es.wikipedia.org/wiki/L%C3%ADnea_'+ str(k) +'_(Metro_de_Madrid)'
    xpath1 = '/html/body/div[3]/div[3]/div[5]/div[1]/table[2]'
    xpath2 = '/html/body/div[3]/div[3]/div[5]/div[1]/table[3]'
    driver.get(url)
    try:
        x1 = zip([j.get_attribute('title') for i in driver.find_element(By.XPATH, xpath1).find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')],[j.text for i in driver.find_element(By.XPATH, xpath1).find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')])
    except:
        x1 = []
    try:
        x2 = zip([j.get_attribute('title') for i in driver.find_element(By.XPATH, xpath2).find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')],[j.text for i in driver.find_element(By.XPATH, xpath2).find_elements(By.XPATH, 'tbody/tr') for j in i.find_elements(By.XPATH, 'td/a')])
    except:
        x2 = []
    driver.quit()
    return [x1,x2]

In [4]:
for i in range(1,11):
    line_dict[i] = table_extractor(i)

C:\Users\jorge\AppData\Local\Temp\ipykernel_9460\2529243415.py:19: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary,options = opciones)


In [7]:
line_dict = {k: [list(filter(lambda x: x[0] not in {'Cercanías Madrid', 'Mantenimiento'}, x)) for x in v] for k,v in line_dict.items()}

In [11]:
with open('raw_metro_data.json', 'w') as fp: # write raw file
    json.dump(line_dict,fp)

In [ ]:
line_node_paths = {}
for k in range(1,11):
    line_node_paths[k] = []
    count = 0
    for i,j in enumerate(line_dict_clean[k]):
        if j[1] or i == 0 or i == len(line_dict_clean[k])-1:
            if j[1]:
                if j[1][0] >= k:
                    line_node_paths[k].append((k,i,count))
                else:
                    val = reduce(lambda x,y : x if x[0] == j[0] else y,line_dict_clean[j[1][0]])
                    line_node_paths[k].append((j[1][0],line_dict_clean[j[1][0]].index(val),count))
            else:
                line_node_paths[k].append((k,i,count))
            count = 0
        count += 1

nodes = []
edges=  []

for i in range(1,11):
    path = line_node_paths[i]
    
    for j in range(len(path)-1):
        node1 = Node((path[j][0],path[j][1]))
        node2 = Node((path[j+1][0],path[j+1][1]))
        if node1 not in nodes:
            nodes.append(node1)
        if node2 not in nodes:
            nodes.append(node2)
        edges.append(Edge((node1,node2),weight=path[j+1][2]))

node_translation = {}
for node in nodes:
    if node not in node_translation:
        node_translation[node] = line_dict_clean[node[0]][node[1]][0]
node_reverse_translation = {v: k for k, v in node_translation.items()}
network = Network(nodes,edges)

In [ ]:
x = network.find_path(node_reverse_translation['Legazpi'],node_reverse_translation['Cuatro Caminos'])
stations = 0
for j,i in enumerate(x):
    print(node_translation[i])
    if j < len(x)-1:

        edge = network.edge_adjacency[i]
        for k in edge:
            if x[j+1] in k:
                stations += k.weight
print(stations)

Legazpi
Plaza Elíptica
Oporto
Casa de Campo
Cuatro Caminos
17


In [ ]:
test = network
test

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Network with 58 Nodes and 94 Edges
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Nodes:
('1.00', '4.00')
('1.00', '5.00')
('1.00', '11.00')
('1.00', '15.00')
('1.00', '16.00')
('1.00', '18.00')
('1.00', '19.00')
('1.00', '23.00')
('1.00', '26.00')
('1.00', '40.00')
('2.00', '0.00')
('2.00', '7.00')
('2.00', '9.00')
('2.00', '10.00')
('2.00', '11.00')
('2.00', '19.00')
('2.00', '21.00')
('2.00', '22.00')
('2.00', '25.00')
('3.00', '0.00')
('3.00', '2.00')
('2.00', '26.00')
('3.00', '5.00')
('3.00', '9.00')
('3.00', '10.00')
('3.00', '14.00')
('3.00', '16.00')
('3.00', '22.00')
('3.00', '23.00')
('4.00', '6.00')
('4.00', '14.00')
('4.00', '15.00')
('4.00', '31.00')
('5.00', '0.00')
('5.00', '10.00')
('5.00', '26.00')
('5.00', '33.00')
('5.00', '41.00')
('6.00', '0.00')
('6.00', '5.00')
('6.00', '11.00')
('6.00', '14.00')
('6.00', '22.00')
('6.00', '24.00')
('6.00', '32.00')
('6.00', '36.00')

In [ ]:
for i,j in enumerate(network.edges):
    try :
        print(j,test.edges[i])
        print(j == test.edges[i])
    except:
        print('exc')
        pass

('1.00', '4.00') <-> ('1.00', '5.00') || 1 ('1.00', '4.00') <-> ('1.00', '5.00') || 1
True
('1.00', '5.00') <-> ('1.00', '11.00') || 6 ('1.00', '5.00') <-> ('1.00', '11.00') || 6
True
('1.00', '11.00') <-> ('1.00', '15.00') || 4 ('1.00', '11.00') <-> ('1.00', '15.00') || 4
True
('1.00', '15.00') <-> ('1.00', '16.00') || 1 ('1.00', '15.00') <-> ('1.00', '16.00') || 1
True
('1.00', '16.00') <-> ('1.00', '18.00') || 2 ('1.00', '16.00') <-> ('1.00', '18.00') || 2
True
('1.00', '18.00') <-> ('1.00', '19.00') || 1 ('1.00', '18.00') <-> ('1.00', '19.00') || 1
True
('1.00', '19.00') <-> ('1.00', '23.00') || 4 ('1.00', '19.00') <-> ('1.00', '23.00') || 4
True
('1.00', '23.00') <-> ('1.00', '26.00') || 3 ('1.00', '23.00') <-> ('1.00', '26.00') || 3
True
('1.00', '26.00') <-> ('1.00', '40.00') || 14 ('1.00', '26.00') <-> ('1.00', '40.00') || 14
True
('2.00', '0.00') <-> ('2.00', '7.00') || 7 ('2.00', '0.00') <-> ('2.00', '7.00') || 7
True
('2.00', '7.00') <-> ('2.00', '9.00') || 2 ('2.00', '7.00'

In [ ]:
network.move_node(network.nodes[0],(0,0))

In [ ]:
len(test.edges)

94